In [8]:
import os
import numpy as np
import rasterio
from rasterio.windows import Window


In [12]:


def remove_black_pixels(input_file, output_file):
    with rasterio.open(input_file) as src:
        # Read the raster data
        raster = src.read(1)
        
        # Identify the rows and columns to keep
        rows = list(set(range(raster.shape[0])) - set(np.where(~raster.any(axis=1))[0]))
        cols = list(set(range(raster.shape[1])) - set(np.where(~raster.any(axis=0))[0]))
        
        # Define a window that includes only the rows and columns with data
        window = Window(min(cols), min(rows), len(cols), len(rows))
        
        # Read the data in the window
        data = src.read(window=window)
        
        # Define the new transform
        transform = src.window_transform(window)
        
        # Write the data to a new file
        with rasterio.open(output_file, 'w', driver='GTiff', height=data.shape[1], width=data.shape[2], count=1, dtype=data.dtype, crs=src.crs, transform=transform) as dst:
            dst.write(data)

def process_directory(input_dir, output_dir):
    # Walk through all files in the input directory
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            # Only process .tif files
            if file.endswith('.tif'):
                # Construct the full file paths
                input_file = os.path.join(root, file)
                relative_path = os.path.relpath(input_file, input_dir)
                output_file = os.path.join(output_dir, relative_path)
                
                # Create the output directory if it doesn't exist
                os.makedirs(os.path.dirname(output_file), exist_ok=True)
                
                # Remove black pixels
                remove_black_pixels(input_file, output_file)

# Call the function
process_directory(r"plotType_Tower", r"plotType_Tower")
